In [1]:
# For compatibility across multiple platforms
import os
import numpy as np
import pandas as pd
from scipy import spatial


# Load files using DictReader in Python
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import cluster
from sklearn import preprocessing
from sklearn.model_selection import KFold
import sklearn


from ultimate.mlp import MLP 

from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
train_filepath = "pubg-finish-placement-prediction/train_V2_clean.csv"

#trainset_file = open(train_filepath,'rU')
trainset = pd.read_csv(train_filepath,index_col=0)

In [17]:
print(len(trainset))

4446965


In [38]:
def feature_engineering_two(moddedTrain):
    moddedTrain['playersJoined'] = moddedTrain.groupby('matchId')['matchId'].transform('count')
    moddedTrain['killsNorm'] = moddedTrain['kills']*((100-moddedTrain['playersJoined'])/100 + 1)
    moddedTrain['damageDealtNorm'] = moddedTrain['damageDealt']*((100-moddedTrain['playersJoined'])/100 + 1)
    moddedTrain['maxPlaceNorm'] = moddedTrain['maxPlace']*((100-moddedTrain['playersJoined'])/100 + 1)
    moddedTrain['matchDurationNorm'] = moddedTrain['matchDuration']*((100-moddedTrain['playersJoined'])/100 + 1)
    moddedTrain['healsandboosts'] = moddedTrain['heals'] + moddedTrain['boosts']
    moddedTrain['totalDistance'] = moddedTrain['rideDistance'] + moddedTrain['walkDistance'] + moddedTrain['swimDistance']
    moddedTrain['killsWithoutMoving'] = ((moddedTrain['kills'] > 0) & (moddedTrain['totalDistance'] == 0))
    moddedTrain['headshot_rate'] = moddedTrain['headshotKills'] / moddedTrain['kills']
    moddedTrain['headshot_rate'] = moddedTrain['headshot_rate'].fillna(0)
    moddedTrain.drop(moddedTrain[moddedTrain['killsWithoutMoving'] == True].index, inplace=True)
    moddedTrain.drop(moddedTrain[moddedTrain['roadKills'] > 8].index, inplace=True)
    

In [28]:
def feature_engineering(is_train=True):
    # When this function is used for the training data, load train_V2.csv :
    if is_train: 
        print("processing train_V2.csv")
        df = pd.read_csv("pubg-finish-placement-prediction/train_V2_clean.csv",index_col=0)
        
        # Only take the samples with matches that have more than 1 player 
        # there are matches with no players or just one player ( those samples could affect our model badly) 
        df = df[df['maxPlace'] > 1]
    
    # When this function is used for the test data, load test_V2.csv :
    else:
        print("processing test_V2.csv")
        df = pd.read_csv(INPUT_DIR + 'test_V2.csv')
        
    # Make a new feature indecating the total distance a player cut :
 
    df['totalDistance'] = df['rideDistance'] + df["walkDistance"] + df["swimDistance"]

    # Process the 'rankPoints' feature by replacing any value of (-1) to be (0) :
    df['rankPoints'] = np.where(df['rankPoints'] <= 0 ,0 , df['rankPoints'])
                           
    

    target = 'winPlacePerc'
    # Get a list of the features to be used
    features = list(df.columns)
    
    # Remove some features from the features list :
    features.remove("Id")
    features.remove("matchId")
    features.remove("groupId")
    features.remove("matchDuration")
    features.remove("matchType")
    
    y = None
    
    # If we are processing the training data, process the target
    # (group the data by the match and the group then take the mean of the target) 
    if is_train: 
        y = np.array(df.groupby(['matchId','groupId'])[target].agg('mean'), dtype=np.float64)
        # Remove the target from the features list :
        features.remove(target)
    
    # Make new features indicating the mean of the features ( grouped by match and group ) :
    print("get group mean feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('mean')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    
    # If we are processing the training data let df_out = the grouped  'matchId' and 'groupId'
    if is_train: df_out = agg.reset_index()[['matchId','groupId']]
    # If we are processing the test data let df_out = 'matchId' and 'groupId' without grouping 
    else: df_out = df[['matchId','groupId']]
    
    # Merge agg and agg_rank (that we got before) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the max value of the features for each group ( grouped by match )
    print("get group max feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('max')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    # Merge the new (agg and agg_rank) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_max", "_max_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the minimum value of the features for each group ( grouped by match )
    print("get group min feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('min')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    # Merge the new (agg and agg_rank) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_min", "_min_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the number of players in each group ( grouped by match )
    print("get group size feature")
    agg = df.groupby(['matchId','groupId']).size().reset_index(name='group_size')
     
    # Merge the group_size feature with df_out :
    df_out = df_out.merge(agg, how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the mean value of each features for each match :
    print("get match mean feature")
    agg = df.groupby(['matchId'])[features].agg('mean').reset_index()
    
    # Merge the new agg with df_out :
    df_out = df_out.merge(agg, suffixes=["", "_match_mean"], how='left', on=['matchId'])
    
    # Make new features indicating the number of groups in each match :
    print("get match size feature")
    agg = df.groupby(['matchId']).size().reset_index(name='match_size')
    
    # Merge the match_size feature with df_out :
    df_out = df_out.merge(agg, how='left', on=['matchId'])
    
    # Drop matchId and groupId
    df_out.drop(["matchId", "groupId"], axis=1, inplace=True)
    
    y = y.tolist()
    

    return df_out,y

In [29]:
df_out,y = feature_engineering()

processing train_V2.csv
get group mean feature
get group max feature
get group min feature
get group size feature
get match mean feature
get match size feature


In [30]:
x_train, x_val, y_train, y_val = train_test_split(df_out, y , test_size=0.2)

# SET UP FOR MODEL ONE

## Select the trainset features for tree one 

In [32]:
x_one = x_train
y_one = y_train

x_one_val = x_val
y_one_val = y_val


#set the dmatrix
#trainset_one_dmatrix = xgb.DMatrix(x_one.values,label=y_train.values,feature_names=x_one.columns)
#valset_one_dmatrix = xgb.DMatrix(x_one_val.values,label=y_one_val.values,feature_names=x_one_val.columns)

In [34]:
trainset_one_dmatrix = xgb.DMatrix(x_one.values,label=y_train,feature_names=x_one.columns)
valset_one_dmatrix = xgb.DMatrix(x_one_val.values,label=y_one_val,feature_names=x_one_val.columns)

In [35]:
x_one.head()

,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,killPlace_mean,killPoints_mean,kills_mean,killStreaks_mean,...,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,totalDistance,match_size
1008717,0.000000,0.000000,33.333333,0.333333,0.000000,0.000000,92.000000,1227.666667,0.000000,0.000000,...,125.435714,0.000000,7.772255,0.010204,0.000000,1033.093939,3.214286,1498.918367,1166.301908,98
672723,0.333333,0.666667,254.066667,2.000000,1.000000,1.000000,20.333333,0.000000,2.333333,1.666667,...,219.021213,0.000000,5.457010,0.020833,0.000000,1257.642292,3.427083,0.000000,1482.120515,96
1940897,0.000000,2.000000,138.420000,0.500000,0.500000,4.000000,31.000000,1234.000000,1.000000,0.500000,...,1196.504167,0.020833,2.019740,0.010417,0.020833,1187.176000,4.291667,1513.843750,2385.699906,96
825965,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,57.000000,1000.000000,0.000000,0.000000,...,43.090638,0.000000,7.266957,0.000000,0.010638,1231.427117,3.765957,1519.372340,1281.784713,94
338466,0.000000,0.000000,171.066667,1.666667,0.333333,0.333333,44.000000,0.000000,0.666667,0.666667,...,20.755914,0.000000,0.014785,0.032258,0.000000,963.941043,3.139785,0.000000,984.711742,93


## Configure tree one

In [36]:
params = {
    "max_depth" : 7,
    "eval_metric" : ["mae"],
    "lambda": 1.1,   
}


In [37]:
iters = 20

tree_one = xgb.train(params, trainset_one_dmatrix, evals=[(trainset_one_dmatrix, "train"),(valset_one_dmatrix, 'val')], num_boost_round = iters)

[23:20:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 252 extra nodes, 0 pruned nodes, max_depth=7
[0]	train-mae:0.184913	val-mae:0.184922
[23:21:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 252 extra nodes, 0 pruned nodes, max_depth=7
[1]	train-mae:0.134009	val-mae:0.134005
[23:22:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 254 extra nodes, 0 pruned nodes, max_depth=7
[2]	train-mae:0.09982	val-mae:0.099816
[23:23:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 252 extra nodes, 0 pruned nodes, max_depth=7
[3]	train-mae:0.077286	val-mae:0.077257
[23:24:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=7
[4]	train-mae:0.062522	val-mae:0.062493
[23:25:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 240 extra nodes, 0 pruned nodes, max_depth=7
[5]	train-mae:0.053126	val-mae:0.053112
[23:26:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 254 extra nodes

xgb.plot_importance(tree_one,importance_type='cover')
xgb.plot_importance(tree_one,importance_type='weight')

# SET UP FOR MODEL TWO

## Select the trainset features for tree two

In [40]:
x_two = x_train
y_two = y_train

x_two_val = x_val
y_two_val = y_val

scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=False).fit(x_two)

In [41]:
scaler.transform(x_two)

array([[-1.        , -1.        , -0.98992342, ...,  0.920351  ,
        -0.69493471,  0.95789474],
       [-0.96825397, -0.95959596, -0.92319629, ..., -1.        ,
        -0.6123272 ,  0.91578947],
       [-1.        , -0.87878788, -0.95815599, ...,  0.93947277,
        -0.37598127,  0.91578947],
       ...,
       [-1.        , -0.96969697, -0.98488513, ..., -1.        ,
        -0.74718103,  0.87368421],
       [-0.97619048, -1.        , -0.97965538, ...,  0.93687894,
        -0.71874774,  0.87368421],
       [-1.        , -0.90909091, -0.98606409, ...,  0.93673303,
        -0.43835967,  0.89473684]])

## Define the NN

In [42]:
# create NN_model
NN_model = Sequential()
NN_model.add(Dense(x_two.shape[1],  input_dim = x_two.shape[1], activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(100, activation='relu'))





# output Layer
NN_model.add(Dense(1, activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 170)               29070     
_________________________________________________________________
dense_2 (Dense)              (None, 136)               23256     
_________________________________________________________________
dense_3 (Dense)              (None, 136)               18632     
_________________________________________________________________
dense_4 (Dense)              (None, 136)               18632     
_________________________________________________________________
dense_5 (Dense)              (None, 136)               18632     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               13700     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total para

In [43]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [45]:
NN_model.fit(x=x_two, y=y_two, batch_size=1000,
             epochs=10, verbose=1, callbacks=callbacks_list,
             validation_split=0.20, validation_data=None, shuffle=True,
             class_weight=None, sample_weight=None, initial_epoch=0,
             steps_per_epoch=None, validation_steps=None)


Train on 1297116 samples, validate on 324279 samples
Epoch 1/10
1297116/1297116 [==============================] - 35s 27us/step - loss: 0.0570 - mean_absolute_error: 0.0570 - val_loss: 0.0546 - val_mean_absolute_error: 0.0546

Epoch 00001: val_loss improved from 0.06112 to 0.05457, saving model to Weights-001--0.05457.hdf5
Epoch 2/10
1297116/1297116 [==============================] - 25s 19us/step - loss: 0.0545 - mean_absolute_error: 0.0545 - val_loss: 0.0513 - val_mean_absolute_error: 0.0513

Epoch 00002: val_loss improved from 0.05457 to 0.05129, saving model to Weights-002--0.05129.hdf5
Epoch 3/10
1297116/1297116 [==============================] - 25s 20us/step - loss: 0.0527 - mean_absolute_error: 0.0527 - val_loss: 0.0538 - val_mean_absolute_error: 0.0538

Epoch 00003: val_loss did not improve from 0.05129
Epoch 4/10
1297116/1297116 [==============================] - 25s 19us/step - loss: 0.0512 - mean_absolute_error: 0.0512 - val_loss: 0.0613 - val_mean_absolute_error: 0.0613



# ENSENBLE MODELS ON VALSET

### Predict - Update - Predict

In [80]:
def gen_predictions(models,labels,num_samples):
    labels = labels[:num_samples]
    
    predictions = []
    
    for model in models:
        data = model['data']
        predictor = model['predictor']

        if model['type'] == "tree":
            matrix = xgb.DMatrix(data,feature_names=data.columns)
            prediction = predictor.predict(matrix)
        if model['type'] == "nn":
            prediction = predictor.predict(data)
        else:
            continue

        predictions.append(prediction)
        
    avg_predictions = [0.0]*len(predictions[0])
    for i in range(len(predictions)):
        
        for j in range(len(predictions[0])):
            avg_predictions[j] += 1.0* predictions[i][j] / len(predictions)

    print(len(avg_predictions))
    return avg_predictions
            

### Ensemble model predictions

In [81]:
models = []
models.append({'data':x_one_val,
                'predictor':NN_model,
                'type':"nn"})

models.append({'data':x_one_val,
                'predictor':tree_one,
                'type':"tree"})



labels = y_val

num_samples = 200000

predictions = gen_predictions(models,labels,num_samples)

405349


In [84]:
print(sklearn.metrics.mean_absolute_error(labels,tree_one.predict(valset_one_dmatrix)))

0.03395361648337445
